In [8]:
import helperfuncs as hp
import numpy as np
import pandas as pd
import sib_ldsc_z as ld
from importlib import reload
import glob
reload(ld)

<module 'sib_ldsc_z' from 'C:\\Users\\Hariharan\\Documents\\git_repos\\SNIPar\\ldsc_reg\\inferz\\sib_ldsc_z.py'>

In [3]:
ldscore_path = "C:/Users/Hariharan/Documents/genoecon_work/snipardata/ldscores/eur_w_ld_chr"

In [ ]:
def M(fh, num=None, N=2, common=False):
    '''Parses .l{N}.M files, split across num chromosomes. See docs/file_formats_ld.txt.'''
    parsefunc = lambda y: [float(z) for z in open(y, 'r').readline().split()]
    suffix = '.l' + str(N) + '.M'
    suffix += '_5_50'

    if num is not None:
        x = np.sum([parsefunc(sub_chr(fh, i) + suffix) for i in get_present_chrs(fh, num+1)], axis=0)
    else:
        x = parsefunc(fh + suffix)

    return np.array(x).reshape((1, len(x)))

In [39]:
# Reading LD Scores

files = glob.glob(f"{ldscore_path}/*[0-9].l2.ldscore.gz")
snp = pd.DataFrame(columns = ["CHR", "SNP", "BP", "CM", "MAF", "L2"])

for file in files:
    snpi = pd.read_csv(file, compression='gzip', sep = "\t")
    M = M()
    snp = pd.concat([snp, snpi])

In [49]:
snp.head()

,CHR,SNP,BP,CM,MAF,L2
0,1,rs12565286,721290,0.410292,0.043536,1.244134
1,1,rs3094315,752566,0.488776,0.163588,6.408547
2,1,rs3131972,752721,0.488868,0.167546,6.282501
3,1,rs3131969,754182,0.489734,0.133245,6.251752
4,1,rs1048488,760912,0.492507,0.203166,6.482785


In [100]:
# Reading Number of Loci
files_M = glob.glob(f"{ldscore_path}/*[0-9].l2.M_5_50")
nloci = pd.DataFrame(columns = ["M", "CHR"])

for file in files_M:
    
    if len(file) == 87:
        chromosome = int(file[-11])
    else:
        chromosome = int(file[-12:-10])
    
    nloci_i = pd.DataFrame({"M" : [M(file[:-10])[0, 0]],
                             "CHR" : [chromosome]})
    
    nloci = pd.concat([nloci, nloci_i])

nloci = nloci.reset_index(drop = True)

In [102]:
snp = snp.merge(nloci, how = "left", on = "CHR")